In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from Classes.Communicator import Communicator

diya_name = "07"
comm = Communicator(diya_name, save_data=False, save_frequency=10, choose_specific_directory=False, verbose=True)

In [2]:
from simple_pid import PID
import numpy as np
from scipy.constants import convert_temperature as conv_temp

class PIDController:
    def __init__(self, kp:float, ki:float, kd:float, setpoint:float, dt:float, output_limits:tuple) -> None:
        # to work, the sample time has to be zero
        self.pid = PID(Kp=kp, Ki=ki, Kd=kd, setpoint=setpoint, sample_time=dt, output_limits=output_limits)

    def get_control_input(self, current_output:float) -> np.ndarray:
        u_HP = self.pid(current_output)
        u_FAN = 1.0
        return np.array([u_HP, u_FAN])
    
dt_disc = 0.5 # s
kp, ki, kd = -2.5, -4.0, 0.0
setpoint = conv_temp(37.0, 'C', 'K') # K
output_limits = (-2, 2) # A
controller = PIDController(kp, ki, kd, setpoint, dt_disc, output_limits)

In [3]:
import datetime

T_amb = conv_temp(25.0, 'C', 'K') # K
R4 = 0.5 # K/W
R5 = 25.0 # K/W
frac = R5/(R4 + R5)

# I_HP = 2.5
x_FAN = 0.0

continue_loop = True
initial_time = datetime.datetime.now()
duration = 10 # seconds

try:
    comm.start()
    meas_df = comm.last_measurement
    Tc = (meas_df["T1_C"][0] + meas_df["T1_C"][0])/2
    Tc_K = conv_temp(Tc, 'C', 'K')
    Tcell_K = frac*Tc_K + (1 - frac)*T_amb
    u = controller.get_control_input(Tcell_K)
    comm.send_control_input(u[0], x_FAN)

    while continue_loop:
        current_time = datetime.datetime.now()
        elapsed_time = current_time - initial_time

        
        if elapsed_time.total_seconds() > duration:
            continue_loop = False

except KeyboardInterrupt:
    print("Interrupted by user")

finally:
    comm.stop()

Communicator started
Message sent: I_HP = -2.0, x_FAN = 0.0
Subscribed to diya07/rx/
ricevuto
Received non-standard message
ricevuto
Received non-standard message
ricevuto
Received non-standard message
ricevuto
Received non-standard message
ricevuto
Received non-standard message
ricevuto
                             T1_C   T2_C   T3_C  x_HP  x_FAN  U_HP_mV  I_HP_mA
2024-11-28 15:19:11.656427  16.31  17.82  40.05   586      0   3524.0   1445.0
ricevuto
                             T1_C   T2_C   T3_C  x_HP  x_FAN  U_HP_mV  I_HP_mA
2024-11-28 15:19:12.596293  16.32  17.81  40.04   584      0   3496.0   1427.0
ricevuto
                             T1_C  T2_C   T3_C  x_HP  x_FAN  U_HP_mV  I_HP_mA
2024-11-28 15:19:13.505050  16.32  17.8  40.03   594      0   3560.0   1573.0
ricevuto
                             T1_C   T2_C   T3_C  x_HP  x_FAN  U_HP_mV  I_HP_mA
2024-11-28 15:19:14.488222  16.33  17.82  40.03   583      0   3492.0   1425.0
ricevuto
                             T1_C   T2_C   T3

In [16]:
import paho.mqtt.client as mqtt

# Define the MQTT broker details
broker = "mqtt.119.ovh"
port = 1883
topic = "diya06"
read_topic = topic + "/rx/"
transmit_topic = topic + "/tx/"
keep_alive = 60

last_message = None

# Callback function when a message is received
def on_message(client, userdata, message):
    last_message = message.payload.decode()
    # print(f"Message received: {last_message} on topic {message.topic}")

    parts = last_message.split()
    if "TMP&U" not in last_message or "Peltier:" not in last_message or "Fan:" not in last_message:
        print("Received incomplete message")
        return
    
    # Parse each component based on its position in the message
    T_1 = float(parts[2])
    T_2 = float(parts[3])
    T_3 = float(parts[4])

    # Parse the 'Peltier' and 'Fan' values by removing their labels
    peltier_value = int(parts[6])
    fan_value = int(parts[8])

    # Print out parsed values
    print("Measurements:", T_1, T_2, T_3)
    print("Peltier Value:", peltier_value)
    print("Fan Value:", fan_value)
    print("-------------------------------------------")

def send_control_input(client:mqtt.Client, x_HP:int, x_FAN:int):
    sign_HP = "+" if x_HP >= 0 else "-"

    # Bound control values
    x_HP = max(min(x_HP, 100), -100)
    x_FAN = max(min(x_FAN, 100), 0)

    # Format control values
    x_HP_formatted = f"{sign_HP}{abs(int(x_HP)):03d}"
    x_FAN_formatted = f"{int(x_FAN):03d}"

    message_str = f"S{x_HP_formatted}{x_FAN_formatted}"

    client.publish(transmit_topic, message_str)
    print(f"Message sent: x_HP = {x_HP}, x_FAN = {x_FAN}")


# Create an MQTT client instance
# cself.client = mqtt.Client(api_version=mqtt.CallbackAPIVersion.VERSION2) ##################################3
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

# Assign the on_message callback
client.on_message = on_message

# Connect to the broker
client.connect(broker, port, keep_alive)

# Subscribe to the specified topic
client.subscribe(read_topic)

(<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 1)

In [17]:
client.loop_forever()

KeyboardInterrupt: 

In [ ]:
I_HP = 0
x_FAN = 0

send_control_input(client, I_HP, x_FAN)

client.loop_start()

Message sent: x_HP = 0, x_FAN = 0


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [ ]:
client.loop_stop()

In [ ]:
client.disconnect()